## Goal
-  load metadata and unweighted unifrac distance matrix into R
-  calculate permanova (vegan::adonis) and dispersion stats (vegan::betadisper); NOTE:values and plots will differ with each rarefaction
-  plot differentially abundant ASVs, KOs, and pathways identified by ANCOM

In [ ]:
#load necessary R packages
library(dplyr)
library(vegan)
library(ggplot2)
library(data.table)
library(ggsignif)
library(ggpubr)

In [ ]:
#file locations
metadata_file = './analysis/TableS1_metadata2.txt'
unifrac_file = './analysis/bdiv/unw_unifrac.tsv'

In [ ]:
#load files as dataframes
metadata = read.table(metadata_file, sep='\t', header=TRUE, check=F)
udm = read.table(unifrac_file, h=T, row.names=1, check=F)

In [ ]:
birds <- subset(metadata, Taxonomy_Genus=="Geospiza")
birds <- droplevels(birds)
bats <- subset(metadata, Taxonomy_Family=="Phyllostomidae")
bats <- droplevels(bats)

In [ ]:
#adonis functions using distance matrix as input

do_adonis_1factor <- function(metadata, dm, factor1) { 
  ix <- intersect(metadata$SampleID,rownames(dm))
  dist <- dm[ix,ix]
  dist <- as.matrix(dist)
  dist <- as.dist(dist)
  y <- adonis(dist~factor1)
  return(y)
}

do_adonis_2factor <- function(metadata, dm, factor1, factor2) { 
  ix <- intersect(metadata$SampleID,rownames(dm))
  dist <- dm[ix,ix]
  dist <- as.matrix(dist)
  dist <- as.dist(dist)
  y <- adonis(dist~factor1*factor2)
  return(y)
}

do_adonis_3factor <- function(metadata, dm, factor1, factor2, factor3) { 
  ix <- intersect(metadata$SampleID,rownames(dm))
  dist <- dm[ix,ix]
  dist <- as.matrix(dist)
  dist <- as.dist(dist)
  y <- adonis(dist~factor1+factor2+factor3)
  return(y)
}

In [ ]:
#run adonis
birdbat_1 <- do_adonis_1factor(metadata, udm, metadata$hematophagous)
birdbat_1

birdbat_2 <- do_adonis_2factor(metadata, udm, metadata$Taxonomy_Family, metadata$hematophagous)
birdbat_2

In [ ]:
bats_3 <- do_adonis_3factor(bats, udm, bats$country, bats$sample_type, bats$hematophagous)
bats_3

In [ ]:
birds_1 <- do_adonis_1factor(birds, udm, birds$hematophagous)
birds_1

In [ ]:
#betadisper function
do_betadisper <- function(metadata,dm,factor1) { 
  ix <- intersect(metadata$SampleID,rownames(dm))
  dist <- dm[ix,ix]
  dist <- as.matrix(dist)
  dist <- as.dist(dist)
  y <- betadisper(dist, factor1, type = "centroid")
  return(y)
}

In [ ]:
#check dispersion
birdbat_feeding <- do_betadisper(metadata,udm, metadata$hematophagous)
anova(birdbat_feeding)
plot(birdbat_feeding,axes=c(1,3))

birdbat_taxon <- do_betadisper(metadata,udm, metadata$Taxonomy_Family)
anova(birdbat_taxon)
plot(birdbat_taxon,axes=c(1,2))

In [ ]:
bats_feeding <- do_betadisper(bats, udm, bats$hematophagous)
anova(bats_feeding)
plot(bats_feeding)

bats_country <- do_betadisper(bats, udm, bats$country)
anova(bats_country)
plot(bats_country)

bats_sample <- do_betadisper(bats, udm, bats$sample_type)
anova(bats_sample)
plot(bats_sample)

In [ ]:
birds_feeding <- do_betadisper(birds, udm, birds$hematophagous)
anova(birds_feeding)
plot(birds_feeding)

## Plot ANCOM results

### ASVs

In [ ]:
# Read in ASVs feature table
asv_filtered_file = './analysis/ancom/ancom_asv_feature-table.txt'
features <- read.table(asv_filtered_file, header=TRUE, sep = '\t', quote = "", comment.char = "", check.names=F)

In [ ]:
#formatting
features.df <- data.frame(t(features[1:dim(features)[1],2:dim(features)[2]]))

colnames(features.df) <- make.names(t(features)[1,1:dim(features)[1]])
features.df<-setDT(features.df, keep.rownames = TRUE)[]

data.df.melt <- melt(features.df)
colnames(data.df.melt)[1] <- "SampleID"

In [ ]:
#need to manually download csv file from Qiime2 visualization of the ASVs and pull taxonomy strings before continuing
ancom_ASV_file = './analysis/ancom/ancom-results.txt'

In [ ]:
ancom_asv = read.table(ancom_ASV_file, sep='\t', header=TRUE, check=F)
sig_asv.df.melt <- subset(data.df.melt, variable %in% ancom_asv$variable)

In [ ]:
setDT(sig_asv.df.melt)[setDT(metadata), Taxonomy_Family := i.Taxonomy_Family, on=c("SampleID")]
setDT(sig_asv.df.melt)[setDT(metadata), hematophagous := i.hematophagous, on=c("SampleID")]
setDT(sig_asv.df.melt)[setDT(ancom_asv), ID_tax := i.ID_tax, on=c("variable")]
sig_asv.df.melt$percent <- sig_asv.df.melt$value/5000*100

top8 <- subset(sig_asv.df.melt, ID_tax=="ASV10_Chitinophagaceae" | ID_tax=="ASV02_Mycoplasmataceae" | ID_tax=="ASV06_Enterobacteriaceae" | 
                    ID_tax=="ASV07_Helicobacteraceae" | ID_tax=="ASV08_Helicobacteraceae" | ID_tax=="ASV12_" | 
                    ID_tax=="ASV15_Peptostreptococcaceae" | ID_tax=="ASV18_Comamonadaceae" | ID_tax=="ASV20_Fusobacteriaceae" |
                    ID_tax=="ASV21_Deferribacteraceae" | ID_tax=="ASV22_Mycoplasmataceae" | ID_tax=="ASV23_Clostridiaceae" | 
                    ID_tax=="ASV24_Peptostreptococcaceae" | ID_tax=="ASV25_Bradyrhizobiaceae" | ID_tax=="ASV26_Lachnospiraceae" | ID_tax=="ASV29_Ruminococcaceae")


In [ ]:
ggplot(top8, aes(x=Taxonomy_Family, y=percent, fill=hematophagous)) +
  geom_boxplot() +
  facet_wrap(~ ID_tax, scales = 'free_y') +
  stat_compare_means(label =  "p.signif", label.x = 1.5, method="wilcox.test") +
  xlab("Order") +
  ylab("Relative Abundance (%)") +
  theme(legend.text = element_text(size = 10)) +
  scale_fill_manual(values=c("royalblue2","firebrick3"), name="Diet", labels=c("not hematophagous", "hematophagous")) +
  theme(axis.text.x=element_text(angle=90,size=12), axis.text.y=element_text(size=12), axis.title.x=element_text(size=14, face='bold'), axis.title.y=element_text(size=14, face='bold')) 

In [ ]:
sigasvs <- sig_asv.df.melt[,c("Taxonomy_Family","hematophagous","variable","value")]
aggdata <- group_by(sigasvs, variable, hematophagous, Taxonomy_Family)
aggdata_summary <- summarize(aggdata, num=n(), mean=mean(value))
write.table(aggdata_summary, "./ancom/ancom_ASVs_mean_abundance.txt", sep='\t')

### KOs

In [ ]:
# Read in picrust KO results
KO_file = './analysis/picrust/cr_d5k.metagenome_predictions.txt'
features <- read.table(KO_file, header=TRUE, sep = '\t', quote = "", comment.char = "", check.names=F)

# Read in ancom detected KOs (downloaded from Qiime2 vizualization of the results)
ancom_KO_file = './analysis/picrust/ancom/ancom_KOs.csv'

In [ ]:
#formatting
features.df <- data.frame(t(features[1:dim(features)[1],2:dim(features)[2]]))

colnames(features.df) <- make.names(t(features)[1,1:dim(features)[1]])
features.df<-setDT(features.df, keep.rownames = TRUE)[]

data.df.melt <- melt(features.df)
colnames(data.df.melt)[1] <- "SampleID"

In [ ]:
ancom_kos <- read.csv(ancom_KO_file, header=T)
sig_kodata.df.melt <- subset(data.df.melt, variable %in% ancom_kos$variable)

setDT(sig_kodata.df.melt)[setDT(metadata), Taxonomy_Family := i.Taxonomy_Family, on=c("SampleID")]
setDT(sig_kodata.df.melt)[setDT(metadata), hematophagous := i.hematophagous, on=c("SampleID")]

sigkos <- sig_kodata.df.melt[,c("Taxonomy_Family","hematophagous","variable","value")]

setDT(sigkos)[setDT(ancom_kos), role := i.role.of.interest, on=c("variable")]
setDT(sigkos)[setDT(ancom_kos), group := i.Host.group, on=c("variable")]

In [ ]:
head(sigkos)

In [ ]:
aggdata <- group_by(sigkos, variable, hematophagous, role, Taxonomy_Family)
aggdata_summary <- summarize(aggdata, num=n(), mean=mean(value))
write.table(aggdata_summary, "./analysis/picrust/ancom/ancom_KOs_mean_abundance.txt", sep='\t')

In [ ]:
head(aggdata_summary)

In [ ]:
# manually rearrange aggdata_summary file and calculate mean difference per KO per host group 
# save new file as 'ancom_KOs_mean_difference.txt'
aggdata_kos <- read.table('./analysis/picrust/ancom/ancom_KOs_mean_difference.txt', header=TRUE, sep = '\t')
                    
ggplot(aggdata_kos, aes(x=variable, y=difference, fill=Taxonomy_Family)) +
  geom_bar(stat="identity", position=position_dodge()) +
  facet_wrap(~role, scales = 'free_y', ncol=6) +
  scale_fill_manual(values=c("darkorange2","slateblue2", "springgreen3")) +
  ylab("Difference in Abundance") +
  xlab("KO") +
  coord_flip() +
  theme(legend.text = element_text(size = 10))

### KEGG pathways

In [ ]:
# Read in picrust pathway results
KEGG_file = './analysis/picrust/cr_d5k.metagenome_predictions.L3.txt'
features <- read.table(KEGG_file, header=TRUE, sep = '\t', quote = "", comment.char = "", check.names=F)

# Read in ancom detected KOs (downloaded from Qiime2 vizualization of the results)
ancom_KEGG_file = './analysis/picrust/ancom/ancom_L3.csv'

In [ ]:
#formatting
features.df <- data.frame(t(features[1:dim(features)[1],2:dim(features)[2]]))

colnames(features.df) <- make.names(t(features)[1,1:dim(features)[1]])
features.df<-setDT(features.df, keep.rownames = TRUE)[]

data.df.melt <- melt(features.df)
colnames(data.df.melt)[1] <- "SampleID"

In [ ]:
ancom_pways <- read.csv(ancom_KEGG_file, header=T)
sig_pdata.df.melt <- subset(data.df.melt, variable %in% ancom_pways$variable)

setDT(sig_pdata.df.melt)[setDT(metadata), Taxonomy_Family := i.Taxonomy_Family, on=c("SampleID")]
setDT(sig_pdata.df.melt)[setDT(metadata), hematophagous := i.hematophagous, on=c("SampleID")]

sigpways <- sig_pdata.df.melt[,c("Taxonomy_Family","hematophagous","variable","value")]
aggdata <- group_by(sigpways, variable, hematophagous, Taxonomy_Family)
aggdata_summary <- summarize(aggdata, num=n(), mean=mean(value))
write.table(aggdata_summary, "./analysis/picrust/ancom/ancom_KEGG_pathways_mean_abundance.txt", sep='\t')

In [ ]:
forplot <- subset(sigpways, variable=="Sporulation" | variable=="RIG.I.like.receptor.signaling.pathway")
forplot <- droplevels(forplot)

ggplot(forplot, aes(x=Taxonomy_Family, y=value, fill=hematophagous)) +
  geom_boxplot() +
  stat_compare_means(label =  "p.signif", label.x = 1.5, method="wilcox.test") +
  facet_wrap(~ variable, scales = 'free_y') + 
  xlab("Host Family") +
  ylab("Predicted KEGG Pathway Abundance") +
  scale_fill_manual(values=c("royalblue2","firebrick3"), name="Diet", labels=c("not hematophagous", "hematophagous")) +
  theme(legend.text = element_text(size = 10)) +
  theme(axis.text.x=element_text(angle=90,size=12), axis.text.y=element_text(size=12), axis.title.x=element_text(size=14, face='bold'), axis.title.y=element_text(size=14, face='bold')) 


## Plot contributions of OTUs to KOs

In [ ]:
#read in greengenes taxonomy table
gg_tax_table_file = "./gg_13_5_otus/taxonomy/97_otu_taxonomy.txt"
ggotus <- read.table(gg_tax_table_file, sep='\t', header=T)

otu_to_ko_file = "./analysis/picrust/ko_metagenome_contributions.tsv"
ko_contrib <- read.table(otu_to_ko_file, sep='\t', header=T)

#pull in taxonomy and other metadata
setDT(ko_contrib)[setDT(ggotus), taxonomy := i.taxonomy, on=c("otuid")] #parse taxonomy string into levels (i.e. Phylum, Class, etc.)
setDT(ko_contrib)[setDT(ancom_kos), role := i.role.of.interest, on=c("variable")]
#label the group(s) in which the KO was differentially abundant
setDT(ko_contrib)[setDT(ancom_kos), group := i.Host.group, on=c("variable")]
setDT(ko_contrib)[setDT(metadata), Taxonomy_Family := i.Taxonomy_Family, on=c("SampleID")]
setDT(ko_contrib)[setDT(metadata), hematophagous := i.hematophagous, on=c("SampleID")]

In [ ]:
birds <- subset(ko_contrib, group=="birds" | group=="bats and birds")
birds <- subset(birds, Taxonomy_Family=="Thraupidae")
bats <- subset(ko_contrib, group=="bats" | group=="bats and birds")
bats <- subset(bats, Taxonomy_Family=="Phyllostomidae")

In [ ]:
a1 <- group_by(birds, variable, Family, role, Taxonomy_Family, hematophagous)
bird_sum <- summarize(a1, num=n(), mean=mean(OTUAbundanceInSample))

a2 <- group_by(bats, variable, Family, role, Taxonomy_Family, hematophagous)
bat_sum <- summarize(a2, num=n(), mean=mean(OTUAbundanceInSample))

#filtered to OTUs with a mean abundance of at least 50 (1% relative abundance given total of 5000)
full <- rbind(filter(bird_sum,mean>=50),filter(bat_sum,mean>=50)) 

In [ ]:
ggplot(full, aes(x=hematophagous, y=mean, fill=Family)) +
  geom_bar(position = "stack", stat = "identity") +
  facet_wrap(~ Taxonomy_Family + role, ncol=5, scales='free_y') + 
  xlab("Hematophagous") +
  ylab("Mean abundance") +
  scale_fill_manual(values=c("#FFFF99" ,"#B15928", "#B2DF8A", "#33A02C" ,"#FB9A99" ,"#E31A1C",
  "#FDBF6F", "#FF7F00" ,"#CAB2D6" ,"#6A3D9A" ,"#A6CEE3","#1F78B4")) +
  theme(legend.text = element_text(size = 10)) +
  theme(axis.text.x=element_text(angle=90,size=12), axis.text.y=element_text(size=12), axis.title.x=element_text(size=14, face='bold'), axis.title.y=element_text(size=14, face='bold')) 
